# MONARCHS Lid Physics Test Harness

This notebook tests the behaviour of **surface melt** in the MONARCHS lid scheme.

The reason to test the lid code is because the 1D ERA5 runs produce lid depths that **only grow** or **reset to zero** 

i.e. the lid never shrinks

According to **Buzzard (2018, GMD & PhD thesis)**:
- **Virtual lids** (≤ 0.1 m) can thin or vanish from surface melt.
- **Real lids** cannot ablate from the surface; they only grow/shrink at the basal interface.

Here we test the MONARCHS implementation using a simplified cell dictionary.


In [10]:
import numpy as np
from monarchs.physics import lid_functions

def make_real_lid_test_cell():
    return {
        "lake": True,
        "lake_depth": 1.0,
        "lid_depth": 0.5,
        "v_lid_depth": 0.0,
        "L_ice": 3.34e5,
        "rho_ice": 917.0,
        "rho_water": 1000.0,
        "k_water": 0.6,
        "k_ice": 2.0,
        "lid_sfc_melt": 0.0,
        "lid_melt_count": 0,
        "has_had_lid": True,
        "lid_temperature": np.ones(20) * 273.15,
        "lake_temperature": np.ones(20) * 274.0,
        "firn_temperature": np.ones(500) * 265.0,
        "Sfrac": np.ones(500),
        "Lfrac": np.zeros(500),
        "firn_depth": np.linspace(0, 35.0, 500),
        "lid_depth_array": np.linspace(0, 0.5, 20),
        "lake_depth_array": np.linspace(0, 1.0, 20),
        "vert_grid": 500,
        "vert_grid_firn": 500,
        "vert_grid_lid": 20,
        "vert_grid_lake": 20,
    }

def make_virtual_lid_test_cell():
    cell = make_real_lid_test_cell()
    cell["lid_depth"] = 0.0
    cell["v_lid_depth"] = 0.05  # 5 cm virtual lid
    return cell

def run_surface_melt_test(cell, steps=6, Q=200.0, dt=3600.0):
    for step in range(steps):
        lid_functions.calc_surface_melt(cell, dt, Q)
        print(
            f"Step {step}: "
            f"v_lid_depth={cell['v_lid_depth']:.3f}, "
            f"lid_depth={cell['lid_depth']:.3f}, "
            f"lake_depth={cell['lake_depth']:.3f}, "
            f"lid_sfc_melt={cell['lid_sfc_melt']}, "
            f"lid_melt_count={cell['lid_melt_count']}"
        )

# using this to hide my user name in file path when division by zero warming happens in test 2  
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


## Test 1: Real lid

We initialise with a 0.5 m **real lid**.

Expectation (Buzzard, 2018):  
- Surface melt should *not* reduce lid thickness.  

In [11]:
print("=== Real lid test ===")
cell_real = make_real_lid_test_cell()
run_surface_melt_test(cell_real)

=== Real lid test ===
Step 0: v_lid_depth=0.000, lid_depth=0.500, lake_depth=1.000, lid_sfc_melt=0.0023508054773767623, lid_melt_count=1
Step 1: v_lid_depth=0.000, lid_depth=0.500, lake_depth=1.000, lid_sfc_melt=0.004701610954753525, lid_melt_count=2
Step 2: v_lid_depth=0.000, lid_depth=0.500, lake_depth=1.000, lid_sfc_melt=0.007052416432130287, lid_melt_count=3
Step 3: v_lid_depth=0.000, lid_depth=0.500, lake_depth=1.000, lid_sfc_melt=0.00940322190950705, lid_melt_count=4
Step 4: v_lid_depth=0.000, lid_depth=0.500, lake_depth=1.000, lid_sfc_melt=0.011754027386883812, lid_melt_count=5
Step 5: v_lid_depth=0.000, lid_depth=0.500, lake_depth=1.000, lid_sfc_melt=0.014104832864260574, lid_melt_count=6


## Test 2: Virtual lid

We initialise with a 0.05 m **virtual lid**.

Expectation (Buzzard, 2018):  
- Virtual lid should thin and eventually vanish under surface melt.  

Observed in MONARCHS:  
- Virtual lid remains fixed.  
- `lid_sfc_melt` becomes NaN due to division by zero (since `lid_depth = 0`).  


In [12]:
print("\n=== Virtual lid test ===")
cell_virtual = make_virtual_lid_test_cell()
run_surface_melt_test(cell_virtual)


=== Virtual lid test ===
Step 0: v_lid_depth=0.050, lid_depth=0.000, lake_depth=1.000, lid_sfc_melt=nan, lid_melt_count=1
Step 1: v_lid_depth=0.050, lid_depth=0.000, lake_depth=1.000, lid_sfc_melt=nan, lid_melt_count=2
Step 2: v_lid_depth=0.050, lid_depth=0.000, lake_depth=1.000, lid_sfc_melt=nan, lid_melt_count=3
Step 3: v_lid_depth=0.050, lid_depth=0.000, lake_depth=1.000, lid_sfc_melt=nan, lid_melt_count=4
Step 4: v_lid_depth=0.050, lid_depth=0.000, lake_depth=1.000, lid_sfc_melt=nan, lid_melt_count=5
Step 5: v_lid_depth=0.050, lid_depth=0.000, lake_depth=1.000, lid_sfc_melt=nan, lid_melt_count=6


## Why `NaN` Appears with Virtual Lid

When we input a **virtual lid** (`lid_depth = 0`, `v_lid_depth > 0`), 
the MONARCHS routine `calc_surface_melt` calculates a conductive flux using `lid_depth` not `v_lid_depth`  

The expression looks like:

```python
kdTdz = (
    (cell["lid_temperature"][0] - 273.15)
    * abs(cell["k_ice"])
    / (cell["lid_depth"] / (cell["vert_grid_lid"]/2) + cell["lid_sfc_melt"])
)
```

- `cell["lid_depth"] = 0.0` (no real lid).  
- `cell["lid_sfc_melt"] = 0.0` at start.  
- Denominator = `(0.0 / (20/2)) + 0.0 = 0.0`.  
- Division by zero → `NaN`.  

This confirms the implementation mismatch:  
- Buzzard (2018): surface melt should act on the **virtual lid**.  
- MONARCHS code: surface melt coded for **real lid**, and fails for a virtual lid i.e. if `lid_depth = 0`.  

## Discussion

Results show:

- **Real lid test**: `lid_depth` remains constant while `lid_sfc_melt` accumulates.  
- **Virtual lid test**: `v_lid_depth` remains constant, `lid_sfc_melt` becomes NaN.  

This indicates that MONARCHS currently:
- Computes surface melt only against `lid_depth` (real lid).  
- Does not apply surface melt to `v_lid_depth` (virtual lid).  
- Produces division-by-zero errors if a virtual lid is present (because lid_depth=0) 

⚠️ This behaviour is inconsistent with Buzzard (2018), where surface melt should act only on the **virtual lid** and could explain why real lid never shrinks in the ERA5 runs
